# Lenet Implementation

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


def main():
    batch_size = 32
    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=True)

    x, label = iter(cifar_train).next()
    print("x:", x.shape, 'label:', label.shape)


if __name__ == '__main__':
    main()


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F


class Lenet5(nn.Module):
    """
    for cifar10 dataset
    """

    def __init__(self):
        super(Lenet5, self).__init__()

        self.conv_unit = nn.Sequential(
            # x: [b,3,32,32] ==> [b,6,x,x]
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            #
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
        )
        # ==> [b,16,5,5]
        # to\do: flatten
        # fc_unit
        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

        # # [b,3,32,32]
        # tmp = torch.randn(2, 3, 32, 32)
        # out = self.conv_unit(tmp)
        # print("conv out: ", out.shape)

        # use Cross Entropy Loss
        # self.criteon = nn.CrossEntropyLoss()

    def forward(self, x):
        """
        : param x: [b,3,32,32]
        : return:
        """
        batchsz = x.size(0)
        # [b,3,32,32] ==> [b,16,5,5]
        x = self.conv_unit(x)
        # [b,16,5,5] ==> [b,16*5*5]
        x = x.view(batchsz, 16*5*5)
        # [b,16*5*5] ==> [b,10]
        logtis = self.fc_unit(x)

        # pred = F.softmax(logtis, dim=1)
        # loss = self.criteon(logtis, y)
        return logtis


def main():
    net = Lenet5()

    # [b,3,32,32]
    tmp = torch.randn(2, 3, 32, 32)
    out = net(tmp)
    print("lenet out: ", out.shape)


if __name__ == "__main__":
    main()


In [4]:
# train
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F


def main():
    batch_size = 256
    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)

    x, label = iter(cifar_train).next()
    print("x:", x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    model = Lenet5().to(device=device)
    print(model)
    criteon = nn.CrossEntropyLoss().to(device=device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    for epoch in range(1000):
        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # x [b,3,32,32] ==>
            # label  [b]
            x, label = x.to(device), label.to(device)
            logits = model(x)
            # logits: [b,10]
            # label: [b]
            loss = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        #
        print(epoch, loss.item())

        model.eval()
        with torch.no_grad():
            # test
            for x, label in cifar_test:
                # [b,3,32,32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b,10]
                total_correct = 0
                total_num = 0
                logits = model(x)
                pred = logits.argmax(dim=1)
                # [b] vs [b] ==> scalar
                total_correct += torch.eq(pred, label).float().sum().item()
                total_num += x.size(0)

            acc = total_correct/total_num

            print(epoch, acc)


if __name__ == "__main__":
    main()

# the model are to be promoted


Files already downloaded and verified
Files already downloaded and verified
x: torch.Size([256, 3, 32, 32]) label: torch.Size([256])


AssertionError: Torch not compiled with CUDA enabled

# ResNet Implementation

In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F


class ResBlk(nn.Module):
    """
    resnet block
    """

    def __init__(self, in_ch, out_ch, stride=1):
        """
        :param in_ch:
        :param out_ch
        """
        super(ResBlk, self).__init__()
        self.conv1 = nn.Conv2d(
            in_ch, out_ch, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(
            out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)

        # self.extra = nn.Sequential()
        # if out_ch != in_ch:
        # [b,in_ch,h,w] ==> [b,out_ch,h,w]
        self.extra = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        """
        :param x: [b,ch,h,w]
        :return
        """
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # short cut
        # element-wise add: [b,ch,h,w]
        # print("x.shape:", x.shape, "out.shape:", out.shape)
        out = self.extra(x) + out

        return out


class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64)
        )
        # followed 4 blocks
        # [b,64,h,w] ==> [b,128,h,w]
        self.blk1 = ResBlk(64, 128, stride=2)
        # [b,128,h,w] ==> [b,256,h,w]
        self.blk2 = ResBlk(128, 256, stride=2)
        # [b,256,h,w] ==> [b,512,h,w]
        self.blk3 = ResBlk(256, 512, stride=2)
        # [b,512,h,w] ==> [b,512,h,w]
        self.blk4 = ResBlk(512, 512, stride=2)

        self.outlayer = nn.Linear(512, 10)

    def forward(self, x):
        """
        :param x:
        :return:
        """
        x = F.relu(self.conv1(x))

        # [b,64,h,w] ==> [b,512,h/16,w/16]
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)
        # print("after conv:", x.shape)  # torch.Size([2, 512, 2, 2])

        # [b,512,h,w] ==> [b,512,1,1]
        x = F.adaptive_avg_pool2d(x, [1, 1])
        # print("after pool:", x.shape)  # torch.Size([2, 512, 1, 1])
        # flatten
        x = x.view(x.size(0), -1)
        x = self.outlayer(x)

        return x


def main():
    # # resblk test
    # tmp = torch.randn(2, 64, 32, 32)
    # blk = ResBlk(64, 128, stride=2)
    # out = blk(tmp)
    # print("resblk out shape:", out.shape)

    x = torch.randn(2, 3, 32, 32)
    model = ResNet18()
    out = model(x)
    print("resnet out shape:", out.shape)


if __name__ == "__main__":
    main()


resnet out shape: torch.Size([2, 10])


In [6]:
# train
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F


def main():
    batch_size = 32
    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=True)

    x, label = iter(cifar_train).next()
    print("x:", x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    model = ResNet18().to(device=device)
    print(model)
    criteon = nn.CrossEntropyLoss().to(device=device)
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    for epoch in range(1000):
        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # x [b,3,32,32] ==>
            # label [b]
            x, label = x.to(device), label.to(device)
            logits = model(x)
            # logits: [b,10]
            # label: [b]

            # backprop
            optimizer.zero_grad()
            loss = criteon(logits, label)
            loss.backward()
            optimizer.step()

        #
        print(epoch, loss.item())

        model.eval()
        with torch.no_grad():
            # test
            for x, label in cifar_test:
                # [b,3,32,32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b,10]
                total_correct = 0
                total_num = 0
                logits = model(x)
                pred = logits.argmax(dim=1)
                # [b] vs [b] ==> scalar
                total_correct += torch.eq(pred, label).float().sum().item()
                total_num += x.size(0)

            acc = total_correct/total_num

            print(epoch, acc)


if __name__ == "__main__":
    main()

# the model are to be promoted


Files already downloaded and verified
Files already downloaded and verified
x: torch.Size([32, 3, 32, 32]) label: torch.Size([32])


AssertionError: Torch not compiled with CUDA enabled